## Семинар 1 Индекс

## Intro

##  Индекс 

Сам по себе индекс - это просто формат хранения данных, он не может осуществлять поиск. Для этого необходимо добавить к нему определенную метрику. Это может быть что-то простое типа булева поиска, а может быть что-то более специфическое или кастомное под задачу.

Давайте посмотрим, что полезного можно вытащить из самого индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе - так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. как эти документы кластеризуются по N тематикам согласно словам, которые в них упоминаются 

## __Задача__: 

**Data:** Коллекция субтитров сезонов Друзьей. Одна серия - один документ.

**To do:** Постройте небольшой модуль поискового движка, который сможет осуществлять поиск по коллекции документов.
На входе запрос и проиндексированная коллекция (в том виде, как посчитаете нужным), на выходе отсортированный по релевантности с запросом список документов коллекции. 

Релизуйте:
    - функцию препроцессинга данных
    - функцию индексирования данных
    - функцию метрики релевантности 
    - собственно, функцию поиска

[download_friends_corpus](https://yadi.sk/d/yVO1QV98CDibpw)

Напоминание про defaultdict: 
> В качестве multiple values словаря рекомендую использовать ``` collections.defaultdict ```                          
> Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [1]:
### _check : в коллекции должно быть около 165 файлов

С помощью обратного индекса посчитайте:  


a) какое слово является самым частотным

b) какое самым редким

c) какой набор слов есть во всех документах коллекции

d) какой сезон был самым популярным у Чендлера? у Моники?

e) кто из главных героев статистически самый популярный? 


In [2]:
import os
import pandas as pd

In [4]:
from collections import OrderedDict, defaultdict, Counter

In [5]:
FriendsOD = OrderedDict()
FriendsOD["Season"] = []
FriendsOD["Episode"] = []
FriendsOD["Title"] = []
FriendsOD["Filepath"] = []

In [6]:
for root, dirs, files in os.walk("friends"):
    for file in files:
        fstr = file.split(" - ")
        FriendsOD["Season"].append(int(fstr[1][:fstr[1].find("x")]))
        FriendsOD["Episode"].append(fstr[1][fstr[1].find("x")+1:])
        FriendsOD["Title"].append(fstr[2][:fstr[2].find(".ru.txt")])
        FriendsOD["Filepath"].append(os.path.join(root, file))

In [7]:
Friends = pd.DataFrame(FriendsOD)

In [8]:
Friends

,Season,Episode,Title,Filepath
0,1,01,The One Where Monica Gets A Roommate,../../friends/Friends - season 1/Friends - 1x0...
1,1,02,The One With The Sonogram At The End,../../friends/Friends - season 1/Friends - 1x0...
2,1,03,The One With The Thumb,../../friends/Friends - season 1/Friends - 1x0...
3,1,04,The One With George Stephanopoulos,../../friends/Friends - season 1/Friends - 1x0...
4,1,05,The One With The East German Laundry Detergent,../../friends/Friends - season 1/Friends - 1x0...
5,1,06,The One With The Butt,../../friends/Friends - season 1/Friends - 1x0...
6,1,07,The One With The Blackout,../../friends/Friends - season 1/Friends - 1x0...
7,1,08,The One Where Nana Dies Twice,../../friends/Friends - season 1/Friends - 1x0...
8,1,09,The One Where Underdog Gets Away,../../friends/Friends - season 1/Friends - 1x0...
9,1,10,The One With The Monkey,../../friends/Friends - season 1/Friends - 1x1...


In [9]:
SerialDict = {}
for index, row in Friends.iterrows():
    Season = int(row["Season"])
    if Season not in SerialDict:
        SerialDict[Season] = {}
    Episode = row["Episode"]
    Title = row["Title"]
    Path = row["Filepath"]
    SerialDict[Season][Episode] = {"title": Title, "path": Path}

In [10]:
import pymorphy2

In [23]:
import re

In [94]:
class SeriesSearch:
    
    def __init__(self, serial_dict):
        self._morph = pymorphy2.MorphAnalyzer()
        self._series_struct = {}
        GUID = 0
        for season in serial_dict:
            for episode in serial_dict[season]:
                self._series_struct[GUID] = {"season": season,
                                             "episode": episode,
                                             "title": serial_dict[season][episode]["title"],
                                             "search_struct": self._index_fp(serial_dict[season][episode]["path"])}
                GUID += 1
        self._serial_struct = {}
        self._general_struct = {}
        first = True
        for GUID in self._series_struct:
            season = self._series_struct[GUID]["season"]
            if season not in self._serial_struct:
                self._serial_struct[season] = {"search_struct": self._series_struct[GUID]["search_struct"]}
            else:
                self._serial_struct[season]["search_struct"] += self._series_struct[GUID]["search_struct"]
            if first:
                self._general_struct = {"search_struct": self._series_struct[GUID]["search_struct"]}
                first = False
            else:
                self._general_struct["search_struct"] += self._series_struct[GUID]["search_struct"]

    def _index_fp(self, filepath):
        with open(filepath, "r", encoding="utf-8-sig") as inf:
            text = inf.read()
        words_list = [self._morph.parse(word)[0].normal_form for word in re.findall(r"[\w\-]+", text.lower()) if word != "-"]
        return Counter(words_list)
    
    def most_frequent(self):
        # a) какое слово является самым частотным
        winner = self._general_struct["search_struct"].most_common(1)
        print(winner[0][0], winner[0][1], sep="\t")
    
    def least_frequent(self):
        # b) какое самым редким
        sort = self._general_struct["search_struct"].most_common(len(self._general_struct["search_struct"]))[::-1]
        least_freq = sort[0][1]
        print(sort[0][0], sort[0][1], sep="\t")
        for elem in sort:
            if elem[1] > least_freq:
                break
            print(elem[0], elem[1], sep="\t")
    
    def found_everywhere(self):
        # c) какой набор слов есть во всех документах коллекции
        first = True
        for GUID in self._series_struct:
            if first:
                Common = set(self._series_struct[GUID]["search_struct"].keys())
                first = False
            else:
                Common = Common.intersection(set(self._series_struct[GUID]["search_struct"].keys()))
        print("\n".join(list(Common)))
    
    def chandler_monica(self):
        # d) какой сезон был самым популярным у Чендлера? у Моники?
        Chandler_val = 0
        Chandler_season = ""
        Monica_val = 0
        Chandler_season = ""
        for season in self._serial_struct:
            cv = self._serial_struct[season]["search_struct"]["чендлера"]
            if cv > Chandler_val:
                Chandler_val = cv
                Chandler_season = season
            mv = self._serial_struct[season]["search_struct"]["моника"]
            if mv > Monica_val:
                Monica_val = mv
                Monica_season = season
        print("Чендлер: ", Chandler_season, " ("+str(Chandler_val)+")")
        print("Моника: ", Monica_season, " ("+str(Monica_val)+")")
    
    def most_popular_character(self):
        # e) кто из главных героев статистически самый популярный?
        Characters = {"чендлера": "Чендлер", "моника": "Моника", "фиби": "Фиби", "джоуя": "Джоуи", "росс": "Росс", "рейчело": "Рейчел"}
        Most_popular = ""
        Most_popular_val = 0
        for character in Characters:
            count = self._general_struct["search_struct"][character]
            if count > Most_popular_val:
                Most_popular_val = count
                Most_popular = character
        print(Characters[Most_popular], Most_popular_val, sep="\t")

In [95]:
%%time

Index = SeriesSearch(SerialDict)

CPU times: user 51.1 s, sys: 35.5 ms, total: 51.1 s
Wall time: 51.1 s


In [96]:
Index.most_frequent() # a) какое слово является самым частотным

я	19933


In [97]:
Index.least_frequent() # b) какое самым редким

гардеробный	1
гардеробный	1
карабасос	1
iii	1
хай-я	1
жульничать	1
асидофилус	1
стефанополис	1
прихожанин	1
mазл	1
папасифакиснуть	1
анастассакиснуть	1
обвенчать	1
понемножку	1
доход	1
пешка	1
суженый	1
потрясающий	1
полетта	1
рыться	1
морить	1
рита	1
послезавтра	1
дикция	1
отличительный	1
оплёвывать	1
шекспировский	1
спонсор	1
ray-ban	1
отвозить	1
пропарить	1
вдвойне	1
duh	1
ман	1
ожерелие	1
жемчужный	1
оскар	1
кросби	1
идти-идти	1
нацист	1
сражаться	1
вуаль	1
128	1
отобедать	1
жёнушка	1
штраф	1
квитанция	1
распивина	1
красивина	1
штрафовать	1
37	1
возвратиться	1
сержант	1
бритый	1
расплыться	1
колготки	1
слип	1
натирать	1
взглянуть-ка	1
наперекосяк	1
пианист	1
твердить	1
превышать	1
телец	1
водолей	1
феноменально	1
сутягин	1
симпатягин	1
полисмен	1
сейчас-сейчас	1
предъявить	1
сумкина	1
несравненный	1
колыбель	1
слишком-то	1
поршировать	1
пугливый	1
экий	1
лесоруб	1
кармашек	1
щебетать	1
миранда	1
кармена	1
трибуна	1
дорасти	1
голливудский	1
разодетый	1
свёкор	1
экономить	1
похищение

комп	1
друганом-	1
вскрыться	1
ловкий	1
процессор	1
питание	1
резервный	1
покусать	1
нелепый	1
самобичевание	1
изнутри	1
зинфандель	1
вместить	1
кэти	1
ти	1
забраковать	1
либерач	1
вырви-глаз	1
аллергичный	1
величина	1
переноситься	1
мерить	1
81-летний	1
ров	1
венис	1
упорный	1
пук	1
биг	1
гуманоид	1
компьютеризированный	1
робот-партнёр	1
нуждаться	1
перерепетировать	1
мtv	1
навязать	1
закадрить	1
флакон	1
папайя	1
экстракт	1
кондиционер	1
рукоделие	1
по-хорошему	1
поторговать	1
1981	1
отрыве-81	1
пупковый	1
дэйтонуть	1
600	1
счесть	1
бестактно	1
притаскивать	1
stream	1
field	1
газетный	1
сфинкс	1
египет	1
фрисенстинлендер	1
опека	1
начитанный	1
ямочка	1
юрист	1
tagalongs	1
cheese	1
chuck	1
позорить	1
эпидемиология	1
молекулярный	1
-этый	1
неразборчивый	1
озеленение	1
средство	1
отремонтировать	1
мешаться	1
штуковина	1
табу	1
возбуждение	1
вступиться	1
докупать	1
неодобрение	1
объесться	1
прикачь	1
ранить	1
плести	1
пряжа	1
ханжа	1
развитый	1
плойка	1
электроприбор	1
листик	1
максимпат

ааааааа	1
вылазиющий	1
предварительный	1
отъехать	1
влияние	1
нууууа	1
мм	1
буффея	1
наю	1
воу-воу-воу	1
протекать	1
пошевелить	1
предельно	1
упаковаться	1
воплощать	1
целовальник	1
по-до-ждать	1
гениальный	1
покинутый	1
отступиться	1
резня	1
покалечить	1
желанный	1
rachel	1
хa	1
ронян	1
невыносимый	1
исправиться	1
по-европейски	1
tower	1
чей-либо	1
апартамент	1
э-м-и-л-ь	1
убого	1
скрыться	1
холодненький	1
story	1
tell	1
gonna	1
was	1
ahh	1
ooh	1
пшеница	1
стог	1
энгус	1
шинковать	1
неполированный	1
пена	1
мереть	1
очистить	1
пассажир	1
подсадка	1
распаковываться	1
хэй	1
нейтральный	1
кривомордый	1
самец	1
гд	1
плотно	1
перемещаться	1
разочарованно	1
мон-мон-моника	1
двояко	1
обьективный	1
хе	1
йеппи	1
рама	1
высадить	1
запереться	1
инут	1
ми-и-	1
тве	1
черреза	1
сас-стаитс	1
ви-ин	1
разда-ач	1
следующа-ая	1
встретииться	1
доргуша	1
заграничный	1
отразиться	1
-эй	1
взаимозаменяемый	1
терапевт	1
уолтхейм	1
прихорашиваться	1
каий-то	1
окружить	1
связующий	1
нерушимый	1
космолёт	1
пункти

timur_nn	1
13340	1
колючка	1
экскурсия	1
класный	1
дествительный	1
скалолазный	1
гимнастика	1
улыбаешся	1
грилла	1
кульминаций	1
кошмарный	1
николсон	1
водонагревтель	1
пробел-пробел	1
интерестнея	1
217	1
слишклма	1
стрейчевой	1
отеец	1
платноический	1
скрытный	1
раздражаться	1
изящно	1
получть	1
трёхочковый	1
двадцатитрёхочковый	1
обманный	1
джо-	1
рукопись	1
останеться	1
насыпать	1
-только	1
-продолжать	1
-мна	1
пыхать	1
старшный	1
пересчиать	1
кельвин	1
поздноватый	1
-посмотреть	1
-у	1
хочеться	1
приготовт	1
отпразновать	1
собираешся	1
приблизительно	1
ээ	1
катаешся	1
будить	1
будеш	1
освободишся	1
-шоколадный	1
тискать	1
оргия	1
головокружительный	1
кейном	1
гражданин	1
класиик	1
статический	1
внушить	1
ведать	1
изревноваться	1
стрип-мама	1
джимборя	1
женщинк	1
витрина	1
пресиоз	1
колыхательный	1
шаго-шаг	1
колышать	1
жамб	1
рон	1
танцовщик	1
откладываться	1
обслужить	1
сигать	1
сопереживать	1
втираться	1
пояснить	1
метить	1
цирюльник	1
сосуд	1
просвечивать	1
шенделира	1
кристал	1


лаймовый	1
мука	1
парень-то	1
неактуальный	1
специфический	1
капризуля	1
нищий	1
благодетельственный	1
сборище	1
идиотски	1
мерзость	1
эбола	1
негатив	1
пиццочка	1
салатик	1
сморкаться	1
сом	1
каджунский	1
порей	1
тайский	1
карпаччио	1
ропера	1
чься-чься	1
свеженазначить	1
легальный	1
сухарь	1
афёра	1
крутовато	1
62	1
мексико	1
обьятия	1
косить	1
оправдывать	1
стейками	1
родриго	1
смахнуть	1
повлечь	1
спонтанность	1
дейсвительный	1
напрыгнуть	1
издалёка	1
оратор	1
гоб	1
обезьян	1
дебра	1
награждение	1
адно	1
бумагоприёмник	1
печать	1
деградационный	1
провокационный	1
знуть	1
двухсосковый	1
врубать	1
порник	1
перебороть	1
матадор	1
страшноватый	1
роджерса	1
кападосец	1
разворот	1
земляной	1
гарден	1
электричка	1
зрелость	1
нарний	1
копировщик	1
ремонтировать	1
многофункциональный	1
чизбургер	1
ку-ку	1
жительство	1
почитай	1
большущий	1
философ	1
почтить	1
хекалза	1
морально	1
доченька	1
подземка	1
причуда	1
чудом	1
самоуверенный	1
расщепить	1
атом	1
плоский	1
узколобый	1
грозный	1
рейча

In [98]:
Index.found_everywhere() # c) какой набор слов есть во всех документах коллекции

с
это
быть
о
как
этот
сказать
и
мочь
ты
на
так
у
не
мы
да
нет
я
она
такой
что
ещё
бы
в
думать
хотеть
мой
если
просто
но
знать
он
тот
весь
а
ну


In [99]:
Index.chandler_monica()  # d) какой сезон был самым популярным у Чендлера? у Моники?

Чендлер:  1  (724)
Моника:  1  (721)


In [100]:
Index.most_popular_character()  # e) кто из главных героев статистически самый популярный?

Росс	1056
